## ML Ops
In this section of the demo, we will utilize Snowpark's Python client-side Dataframe API as well as the Snowpark server-side runtime to create an **ML ops pipeline**.  We will take the functions created by the Data Scientist and ML Engineer and create a set of functions that can be easily automated with the company's orchestration tools. 

The ML Engineer must create a pipeline to **automate deployment** of models and batch predictions where the business users can consume them easily from an ML-powered front-end application like Streamlit.  The predictions must be accompanied by an explanation of which features were most impactful for the prediction.  

Most importantly no data should leave Snowflake.  The entire end-to-end workflow should be pushed-down to run where the data sits.

Input: Data in `trips` table.  Feature engineering, train, predict functions from data scientist.  
Output: Automatable pipeline of feature engineering, train, predict.

### 1. Load  credentials and connect to Snowflake

In [ ]:
from dags.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect('./include/state.json')

### 1. Setup Training and Inference Pipeline

We will generate a unique identifier which we will use to provide lineage across all components of the pipeline.

In [ ]:
from snowflake.snowpark import functions as F
import uuid
run_date='2020_01_01'
model_id = str(uuid.uuid1()).replace('-', '_')

state_dict.update({'model_id': model_id})
state_dict.update({'run_date': run_date})
state_dict.update({'weather_database_name': 'WEATHER_NYC'})
state_dict.update({'trips_table_name': 'TRIPS',
                   'holiday_table_name': 'HOLIDAYS',
                   'weather_table_name': state_dict['weather_database_name']+'.ONPOINT_ID.HISTORY_DAY',
                   'weather_view_name': 'WEATHER_NYC_VW',
                   'feature_table_name' : 'FEATURES_'+model_id,
                   'pred_table_name': 'PREDS_'+model_id,
                   'eval_table_name': 'EVALS_'+model_id,
                   'forecast_table_name': 'FORECAST_'+model_id,
                   'forecast_steps': 30,
                   'train_udf_name': 'station_train_predict_udf',
                   'train_func_name': 'station_train_predict_func',
                   'eval_udf_name': 'eval_model_output_udf',
                   'eval_func_name': 'eval_model_func',
                   'model_stage_name': 'MODEL_STAGE',
                  })

import json
with open('./include/state.json', 'w') as sdf:
    json.dump(state_dict, sdf)

We will deploy the model training and inference as a permanent [Python Snowpark User-Defined Function (UDF)](https://docs.snowflake.com/en/LIMITEDACCESS/snowpark-python.html#creating-user-defined-functions-udfs-for-dataframes). This will make the function available to not only our automated training/inference pipeline but also to any users needing the function for manually generated predictions.  
  
As a permanent function we will need a staging area.

In [ ]:
session.sql('CREATE STAGE IF NOT EXISTS ' + state_dict['model_stage_name']).collect()

For production we need to be able to reproduce results.  The `trips` table will change as new data is loaded each month so we need a point-in-time snapshot.  Snowflake [Zero-Copy Cloning](https://docs.snowflake.com/en/sql-reference/sql/create-clone.html) allows us to do this with copy-on-write features so we don't have multiple copies of the same data.  We will create a unique ID to identify each training/inference run as well as the features and predictions generated.  We can use [object tagging](https://docs.snowflake.com/en/user-guide/object-tagging.html) to tag each object with the `model_id` as well.

In [ ]:
clone_table_name = 'TRIPS_CLONE_'+state_dict["run_date"]
state_dict.update({"clone_table_name":clone_table_name})

_ = session.sql('CREATE OR REPLACE TABLE '+clone_table_name+" CLONE "+state_dict["trips_table_name"]).collect()
_ = session.sql('CREATE TAG IF NOT EXISTS model_id_tag').collect()
_ = session.sql("ALTER TABLE "+clone_table_name+" SET TAG model_id_tag = '"+state_dict["model_id"]+"'").collect()

We will start by importing the functions created by the ML Engineer.

In [ ]:
from dags.mlops_pipeline import materialize_holiday_table
from dags.mlops_pipeline import subscribe_to_weather_data
from dags.mlops_pipeline import create_weather_view
from dags.mlops_pipeline import deploy_pred_train_udf
from dags.mlops_pipeline import deploy_eval_udf
from dags.mlops_pipeline import create_forecast_table
from dags.mlops_pipeline import create_feature_table
from dags.mlops_pipeline import train_predict
from dags.mlops_pipeline import evaluate_station_model
from dags.mlops_pipeline import flatten_tables

The pipeline will be orchestrated by our companies orchestration framework but we will test the steps here.

In [ ]:
holiday_table_name = materialize_holiday_table(session=session, 
                                               holiday_table_name=state_dict['holiday_table_name'])

In [ ]:
weather_database_name = subscribe_to_weather_data(session=session, 
                                                  weather_database_name=state_dict['weather_database_name'], 
                                                  weather_listing_id=state_dict['weather_listing_id'])

In [ ]:
weather_view_name = create_weather_view(session, 
                                        weather_table_name=state_dict['weather_table_name'],
                                        weather_view_name=state_dict['weather_view_name'])

In [ ]:
model_udf_name = deploy_pred_train_udf(session=session, 
                                       udf_name=state_dict['train_udf_name'],
                                       function_name=state_dict['train_func_name'],
                                       model_stage_name=state_dict['model_stage_name'])

In [ ]:
eval_udf_name = deploy_eval_udf(session=session, 
                                udf_name=state_dict['eval_udf_name'],
                                function_name=state_dict['eval_func_name'],
                                model_stage_name=state_dict['model_stage_name'])

In [ ]:
feature_table_name = create_feature_table(session, 
                                          trips_table_name=state_dict['clone_table_name'], 
                                          holiday_table_name=state_dict['holiday_table_name'], 
                                          weather_view_name=state_dict['weather_view_name'],
                                          feature_table_name=state_dict['feature_table_name'])

_ = session.sql("ALTER TABLE "+feature_table_name+" SET TAG model_id_tag = '"+state_dict["model_id"]+"'").collect()


In [ ]:
forecast_table_name = create_forecast_table(session, 
                                            trips_table_name=state_dict['trips_table_name'],
                                            holiday_table_name=state_dict['holiday_table_name'], 
                                            weather_view_name=state_dict['weather_view_name'], 
                                            forecast_table_name=state_dict['forecast_table_name'],
                                            steps=state_dict['forecast_steps'])

_ = session.sql("ALTER TABLE "+forecast_table_name+" SET TAG model_id_tag = '"+state_dict["model_id"]+"'").collect()


In [ ]:
session.use_warehouse(state_dict['compute_parameters']['train_warehouse'])

In [ ]:
pred_table_name = train_predict(session, 
                                station_train_pred_udf_name=state_dict['train_udf_name'], 
                                feature_table_name=state_dict['feature_table_name'], 
                                forecast_table_name=state_dict['forecast_table_name'],
                                pred_table_name=state_dict['pred_table_name'])

_ = session.sql("ALTER TABLE "+pred_table_name+" SET TAG model_id_tag = '"+state_dict["model_id"]+"'").collect()


In [ ]:
#session.sql('ALTER WAREHOUSE IF EXISTS '+state_dict['compute_parameters']['train_warehouse']+' SUSPEND').collect()

In [ ]:
session.use_warehouse(state_dict['compute_parameters']['default_warehouse'])

In [ ]:
eval_table_name = evaluate_station_model(session, 
                                         run_date=state_dict['run_date'], 
                                         eval_model_udf_name=state_dict['eval_udf_name'], 
                                         pred_table_name=state_dict['pred_table_name'], 
                                         eval_table_name=state_dict['eval_table_name'])

_ = session.sql("ALTER TABLE "+eval_table_name+" SET TAG model_id_tag = '"+state_dict["model_id"]+"'").collect()


In [ ]:
flat_pred_table, flat_forecast_table, flat_eval_table = flatten_tables(session, 
                                                                       pred_table_name, 
                                                                       forecast_table_name, 
                                                                       eval_table_name)
state_dict['flat_pred_table'] = flat_pred_table
state_dict['flat_forecast_table'] = flat_forecast_table
state_dict['flat_eval_table'] = flat_eval_table

_ = session.sql("ALTER TABLE "+flat_pred_table+" SET TAG model_id_tag = '"+state_dict["model_id"]+"'").collect()
_ = session.sql("ALTER TABLE "+flat_forecast_table+" SET TAG model_id_tag = '"+state_dict["model_id"]+"'").collect()
_ = session.sql("ALTER TABLE "+flat_eval_table+" SET TAG model_id_tag = '"+state_dict["model_id"]+"'").collect()


In [ ]:
session.close()

## 2. Consolidate Ingest, Training, Inference and Evaluation

In [ ]:
%%writefile dags/mlops_tasks.py

def snowpark_database_setup(state_dict:dict)-> dict: 
    import snowflake.snowpark.functions as F
    from dags.snowpark_connection import snowpark_connect
    from dags.elt import reset_database

    session, _ = snowpark_connect('./include/state.json')
    reset_database(session=session, state_dict=state_dict, prestaged=True)

    _ = session.sql('CREATE STAGE '+state_dict['model_stage_name']).collect()
    _ = session.sql('CREATE TAG model_id_tag').collect()

    session.close()

    return state_dict

def incremental_elt_task(state_dict: dict, files_to_download:list)-> dict:
    from dags.ingest import incremental_elt
    from dags.snowpark_connection import snowpark_connect

    session, _ = snowpark_connect()

    print('Ingesting '+str(files_to_download))
    download_base_url=state_dict['connection_parameters']['download_base_url']

    _ = session.use_warehouse(state_dict['compute_parameters']['load_warehouse'])

    _ = incremental_elt(session=session, 
                        state_dict=state_dict, 
                        files_to_ingest=files_to_download,
                        download_base_url=download_base_url,
                        use_prestaged=True)

    #_ = session.sql('ALTER WAREHOUSE IF EXISTS '+state_dict['compute_parameters']['load_warehouse']+\
    #                ' SUSPEND').collect()

    session.close()
    return state_dict

def initial_bulk_load_task(state_dict:dict)-> dict:
    from dags.ingest import bulk_elt
    from dags.snowpark_connection import snowpark_connect

    session, _ = snowpark_connect()

    _ = session.use_warehouse(state_dict['compute_parameters']['load_warehouse'])

    print('Running initial bulk ingest from '+state_dict['connection_parameters']['download_base_url'])
    
    _ = bulk_elt(session=session, 
                 state_dict=state_dict, 
                 download_base_url=state_dict['connection_parameters']['download_base_url'],
                 use_prestaged=True)

    #_ = session.sql('ALTER WAREHOUSE IF EXISTS '+state_dict['compute_parameters']['load_warehouse']+\
    #                ' SUSPEND').collect()

    session.close()
    return state_dict

def materialize_holiday_task(state_dict: dict)-> dict:
    from dags.snowpark_connection import snowpark_connect
    from dags.mlops_pipeline import materialize_holiday_table

    print('Materializing holiday table.')
    session, _ = snowpark_connect()

    _ = materialize_holiday_table(session=session, 
                                  holiday_table_name=state_dict['holiday_table_name'])

    session.close()
    return state_dict

def subscribe_to_weather_data_task(state_dict: dict)-> dict:
    from dags.snowpark_connection import snowpark_connect
    from dags.mlops_pipeline import subscribe_to_weather_data

    print('Subscribing to weather data')
    session, _ = snowpark_connect()

    _ = subscribe_to_weather_data(session=session, 
                                  weather_database_name=state_dict['weather_database_name'], 
                                  weather_listing_id=state_dict['weather_listing_id'])
    session.close()
    return state_dict

def create_weather_view_task(state_dict: dict)-> dict:
    from dags.snowpark_connection import snowpark_connect
    from dags.mlops_pipeline import create_weather_view

    print('Creating weather view')
    session, _ = snowpark_connect()

    _ = create_weather_view(session=session,
                            weather_table_name=state_dict['weather_table_name'],
                            weather_view_name=state_dict['weather_view_name'])
    session.close()
    return state_dict
    
def deploy_model_udf_task(state_dict:dict)-> dict:
    from dags.snowpark_connection import snowpark_connect
    from dags.mlops_pipeline import deploy_pred_train_udf

    print('Deploying station model')
    session, _ = snowpark_connect()

    _ = session.sql('CREATE STAGE IF NOT EXISTS ' + state_dict['model_stage_name']).collect()

    _ = deploy_pred_train_udf(session=session, 
                              udf_name=state_dict['train_udf_name'],
                              function_name=state_dict['train_func_name'],
                              model_stage_name=state_dict['model_stage_name'])
    session.close()
    return state_dict

def deploy_eval_udf_task(state_dict:dict)-> dict:
    from dags.snowpark_connection import snowpark_connect
    from dags.mlops_pipeline import deploy_eval_udf

    print('Deploying station model')
    session, _ = snowpark_connect()

    _ = session.sql('CREATE STAGE IF NOT EXISTS ' + state_dict['model_stage_name']).collect()

    _ = deploy_eval_udf(session=session, 
                        udf_name=state_dict['eval_udf_name'],
                        function_name=state_dict['eval_func_name'],
                        model_stage_name=state_dict['model_stage_name'])
    session.close()
    return state_dict

def generate_feature_table_task(state_dict:dict, 
                                holiday_state_dict:dict, 
                                weather_state_dict:dict)-> dict:
    from dags.snowpark_connection import snowpark_connect
    from dags.mlops_pipeline import create_feature_table

    print('Generating features for all stations.')
    session, _ = snowpark_connect()

    session.use_warehouse(state_dict['compute_parameters']['fe_warehouse'])

    _ = session.sql("CREATE OR REPLACE TABLE "+state_dict['clone_table_name']+\
                    " CLONE "+state_dict['trips_table_name']).collect()
    _ = session.sql("ALTER TABLE "+state_dict['clone_table_name']+\
                    " SET TAG model_id_tag = '"+state_dict['model_id']+"'").collect()

    _ = create_feature_table(session, 
                             trips_table_name=state_dict['clone_table_name'], 
                             holiday_table_name=state_dict['holiday_table_name'], 
                             weather_view_name=state_dict['weather_view_name'],
                             feature_table_name=state_dict['feature_table_name'])

    _ = session.sql("ALTER TABLE "+state_dict['feature_table_name']+\
                    " SET TAG model_id_tag = '"+state_dict['model_id']+"'").collect()

    session.close()
    return state_dict

def generate_forecast_table_task(state_dict:dict, 
                                 holiday_state_dict:dict, 
                                 weather_state_dict:dict)-> dict: 
    from dags.snowpark_connection import snowpark_connect
    from dags.mlops_pipeline import create_forecast_table

    print('Generating forecast features.')
    session, _ = snowpark_connect()

    _ = create_forecast_table(session, 
                              trips_table_name=state_dict['trips_table_name'],
                              holiday_table_name=state_dict['holiday_table_name'], 
                              weather_view_name=state_dict['weather_view_name'], 
                              forecast_table_name=state_dict['forecast_table_name'],
                              steps=state_dict['forecast_steps'])

    _ = session.sql("ALTER TABLE "+state_dict['forecast_table_name']+\
                    " SET TAG model_id_tag = '"+state_dict['model_id']+"'").collect()

    session.close()
    return state_dict

def bulk_train_predict_task(state_dict:dict, 
                            feature_state_dict:dict, 
                            forecast_state_dict:dict)-> dict: 
    from dags.snowpark_connection import snowpark_connect
    from dags.mlops_pipeline import train_predict

    state_dict = feature_state_dict

    print('Running bulk training and forecast.')
    session, _ = snowpark_connect()

    session.use_warehouse(state_dict['compute_parameters']['train_warehouse'])

    pred_table_name = train_predict(session, 
                                    station_train_pred_udf_name=state_dict['train_udf_name'], 
                                    feature_table_name=state_dict['feature_table_name'], 
                                    forecast_table_name=state_dict['forecast_table_name'],
                                    pred_table_name=state_dict['pred_table_name'])

    _ = session.sql("ALTER TABLE "+state_dict['pred_table_name']+\
                    " SET TAG model_id_tag = '"+state_dict['model_id']+"'").collect()
    #_ = session.sql('ALTER WAREHOUSE IF EXISTS '+state_dict['compute_parameters']['train_warehouse']+\
    #                ' SUSPEND').collect()

    session.close()
    return state_dict

def eval_station_models_task(state_dict:dict, 
                             pred_state_dict:dict,
                             run_date:str)-> dict:

    from dags.snowpark_connection import snowpark_connect
    from dags.mlops_pipeline import evaluate_station_model

    print('Running eval UDF for model output')
    session, _ = snowpark_connect()

    eval_table_name = evaluate_station_model(session, 
                                             run_date=run_date, 
                                             eval_model_udf_name=state_dict['eval_udf_name'], 
                                             pred_table_name=state_dict['pred_table_name'], 
                                             eval_table_name=state_dict['eval_table_name'])

    _ = session.sql("ALTER TABLE "+state_dict['eval_table_name']+\
                    " SET TAG model_id_tag = '"+state_dict['model_id']+"'").collect()
    session.close()
    return state_dict                                               

def flatten_tables_task(pred_state_dict:dict, state_dict:dict)-> dict:
    from dags.snowpark_connection import snowpark_connect
    from dags.mlops_pipeline import flatten_tables

    print('Flattening tables for end-user consumption.')
    session, _ = snowpark_connect()

    flat_pred_table, flat_forecast_table, flat_eval_table = flatten_tables(session,
                                                                           pred_table_name=state_dict['pred_table_name'], 
                                                                           forecast_table_name=state_dict['forecast_table_name'], 
                                                                           eval_table_name=state_dict['eval_table_name'])
    state_dict['flat_pred_table'] = flat_pred_table
    state_dict['flat_forecast_table'] = flat_forecast_table
    state_dict['flat_eval_table'] = flat_eval_table

    _ = session.sql("ALTER TABLE "+flat_pred_table+" SET TAG model_id_tag = '"+state_dict['model_id']+"'").collect()
    _ = session.sql("ALTER TABLE "+flat_forecast_table+" SET TAG model_id_tag = '"+state_dict['model_id']+"'").collect()
    _ = session.sql("ALTER TABLE "+flat_eval_table+" SET TAG model_id_tag = '"+state_dict['model_id']+"'").collect()

    return state_dict


In [ ]:
%%writefile dags/mlops_setup_pipeline.py
from dags.mlops_tasks import snowpark_database_setup
from dags.mlops_tasks import initial_bulk_load_task
from dags.mlops_tasks import materialize_holiday_task
from dags.mlops_tasks import subscribe_to_weather_data_task
from dags.mlops_tasks import create_weather_view_task
from dags.mlops_tasks import deploy_model_udf_task
from dags.mlops_tasks import deploy_eval_udf_task
from dags.mlops_tasks import generate_feature_table_task
from dags.mlops_tasks import generate_forecast_table_task
from dags.mlops_tasks import bulk_train_predict_task
from dags.mlops_tasks import eval_station_models_task 
from dags.mlops_tasks import flatten_tables_task

def citibikeml_setup_taskflow(run_date:str):
    """
    End to end Snowflake ML Demo
    """
    import uuid
    import json

    with open('./include/state.json') as sdf:
        state_dict = json.load(sdf)
    
    model_id = str(uuid.uuid1()).replace('-', '_')

    state_dict.update({'model_id': model_id})
    state_dict.update({'run_date': run_date})
    state_dict.update({'weather_database_name': 'WEATHER_NYC'})
    state_dict.update({'load_table_name': 'RAW_',
                       'trips_table_name': 'TRIPS',
                       'load_stage_name': 'LOAD_STAGE',
                       'model_stage_name': 'MODEL_STAGE',
                       'weather_table_name': state_dict['weather_database_name']+'.ONPOINT_ID.HISTORY_DAY',
                       'weather_view_name': 'WEATHER_NYC_VW',
                       'holiday_table_name': 'HOLIDAYS',
                       'clone_table_name': 'CLONE_'+model_id,
                       'feature_table_name' : 'FEATURE_'+model_id,
                       'pred_table_name': 'PRED_'+model_id,
                       'eval_table_name': 'EVAL_'+model_id,
                       'forecast_table_name': 'FORECAST_'+model_id,
                       'forecast_steps': 30,
                       'train_udf_name': 'station_train_predict_udf',
                       'train_func_name': 'station_train_predict_func',
                       'eval_udf_name': 'eval_model_output_udf',
                       'eval_func_name': 'eval_model_func'
                      })
    
    #Task order - one-time setup
    setup_state_dict = snowpark_database_setup(state_dict)
    load_state_dict = initial_bulk_load_task(setup_state_dict)
    holiday_state_dict = materialize_holiday_task(setup_state_dict)
    subscribe_state_dict = subscribe_to_weather_data_task(setup_state_dict)
    weather_state_dict = create_weather_view_task(subscribe_state_dict)
    model_udf_state_dict = deploy_model_udf_task(setup_state_dict)
    eval_udf_state_dict = deploy_eval_udf_task(setup_state_dict)
    feature_state_dict = generate_feature_table_task(load_state_dict, holiday_state_dict, weather_state_dict) 
    foecast_state_dict = generate_forecast_table_task(load_state_dict, holiday_state_dict, weather_state_dict)
    pred_state_dict = bulk_train_predict_task(model_udf_state_dict, feature_state_dict, foecast_state_dict)
    eval_state_dict = eval_station_models_task(eval_udf_state_dict, pred_state_dict, run_date)  
    state_dict = flatten_tables_task(pred_state_dict, eval_state_dict)

    return state_dict

In [ ]:
%%writefile dags/mlops_monthly_pipeline.py

from dags.mlops_tasks import incremental_elt_task
from dags.mlops_tasks import generate_feature_table_task
from dags.mlops_tasks import generate_forecast_table_task
from dags.mlops_tasks import bulk_train_predict_task
from dags.mlops_tasks import eval_station_models_task 
from dags.mlops_tasks import flatten_tables_task

def citibikeml_monthly_taskflow(files_to_download:list, run_date:str):
    """
    End to end Snowflake ML Demo
    """
    import uuid
    import json

    with open('./include/state.json') as sdf:
        state_dict = json.load(sdf)
    
    model_id = str(uuid.uuid1()).replace('-', '_')

    state_dict.update({'model_id': model_id})
    state_dict.update({'run_date': run_date})
    state_dict.update({'weather_database_name': 'WEATHER_NYC'})
    state_dict.update({'load_table_name': 'RAW_',
                       'trips_table_name': 'TRIPS',
                       'load_stage_name': 'LOAD_STAGE',
                       'model_stage_name': 'MODEL_STAGE',
                       'weather_table_name': state_dict['weather_database_name']+'.ONPOINT_ID.HISTORY_DAY',
                       'weather_view_name': 'WEATHER_NYC_VW',
                       'holiday_table_name': 'HOLIDAYS',
                       'clone_table_name': 'CLONE_'+model_id,
                       'feature_table_name' : 'FEATURE_'+model_id,
                       'pred_table_name': 'PRED_'+model_id,
                       'eval_table_name': 'EVAL_'+model_id,
                       'forecast_table_name': 'FORECAST_'+model_id,
                       'forecast_steps': 30,
                       'train_udf_name': 'station_train_predict_udf',
                       'train_func_name': 'station_train_predict_func',
                       'eval_udf_name': 'eval_model_output_udf',
                       'eval_func_name': 'eval_model_func'
                      })
    
    #Task order - monthlyl incremental
    incr_state_dict = incremental_elt_task(state_dict, files_to_download)
    feature_state_dict = generate_feature_table_task(incr_state_dict, incr_state_dict, incr_state_dict) 
    forecast_state_dict = generate_forecast_table_task(incr_state_dict, incr_state_dict, incr_state_dict)
    pred_state_dict = bulk_train_predict_task(feature_state_dict, feature_state_dict, forecast_state_dict)
    eval_state_dict = eval_station_models_task(pred_state_dict, pred_state_dict, run_date)
    state_dict = flatten_tables_task(pred_state_dict, eval_state_dict)

    return state_dict

In [ ]:
from dags.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect('./include/state.json')

In [ ]:
%%time
from dags.mlops_setup_pipeline import citibikeml_setup_taskflow

state_dict = citibikeml_setup_taskflow(run_date='2020_01_01')

In [ ]:
%%time
from dags.mlops_monthly_pipeline import citibikeml_monthly_taskflow

state_dict = citibikeml_monthly_taskflow(files_to_download = ['202001-citibike-tripdata.csv.zip'], 
                                         run_date='2020_02_01')

In [ ]:
%%time
state_dict = citibikeml_monthly_taskflow(files_to_download = ['202002-citibike-tripdata.csv.zip'], 
                                         run_date='2020_03_01')

In [ ]:
%%time
state_dict = citibikeml_monthly_taskflow(files_to_download = ['202003-citibike-tripdata.csv.zip'], 
                                         run_date='2020_04_01')

In [ ]:
%%time
state_dict = citibikeml_monthly_taskflow(files_to_download = ['202004-citibike-tripdata.csv.zip'], 
                                         run_date='2020_05_01')

In [ ]:
%%time
state_dict = citibikeml_monthly_taskflow(files_to_download = ['202005-citibike-tripdata.csv.zip'], 
                                         run_date='2020_06_01')

In [ ]:
%%time
state_dict = citibikeml_monthly_taskflow(files_to_download = ['202006-citibike-tripdata.csv.zip'], 
                                         run_date='2020_07_01')

In [ ]:
%%time
state_dict = citibikeml_monthly_taskflow(files_to_download = ['202007-citibike-tripdata.csv.zip'], 
                                         run_date='2020_08_01')

In [ ]:
%%time
state_dict = citibikeml_monthly_taskflow(files_to_download = ['202008-citibike-tripdata.csv.zip'], 
                                         run_date='2020_09_01')

In [ ]:
session.close()